# 3 A PAT application
> In 2004, the Food and Drug Administration (FDA) released a guidance to encourage innovation in pharmaceutical development, manufacturing and quality assurance. The framework, called Process Analytical Technology (PAT), combines MultiVariate Data Analysis (MVDA), Design of Experiments (DoE) and process analytical chemistry methods such as UV, IR, NIR,
NMR (fast, precise, online and preferably non-invasive methods).


>Here, we use PCA and PLS, two classic MVDA approaches to assess the feasibility of using NIR to measure a critical quality component of a wood product, the composition of wood ber. In the rst part, we will perform unsupervised data exploration with PCA. In the second part, we will run PLS, a supervised learning algorithm, to quantify the outputs of interest (i.e. wood composition).

##3.1 Description of the process
>Sawdust from industrial sawing of birch, pine and spruce is mixed at different ratios to manufacture a final wood product. The NIR spectra for 15 samples of known composition of the three different types of wood were used to train the model. Each sample was tested twice (resulting in 30 rows) and NIR spectra data was collected in the range between 1100-2300 nm. Thus, the input X-table is (30 rows)x(1201 columns) and the output Y-table
is (30 rows)x(3 columns), where the three outputs/compositions are recorded as percent proportion of spruce, pine and birch. Data for the training of the model are in the file **sawdust train.xlsx**


>In order to assess the accuracy of the model developed, we have a test data set of NIR data (X) and proportions Y dierent than the training data set. The test data set is not "seen" by the model during training. The proportions in the test set were selected to be
different than those in the train set to assess the predictive power of the model under realistic conditions. There were 12 samples measured twice, therefore the number of rows in the X and Y-tables is 24. The test data are in the file **sawdust pred.xlsx**

##3.2 Task 1: Unsupervised learning, PCA of NIR data
> A. Generate the plot of all the training set raw NIR data.

> B. Pre-process the NIR data and perform PCA for 5 components.

> C. Plot the cumulative explained variance versus the number of components. How many
components do we need to capture 99% of the variance?

> D. Plot the PCA scores for the rst two components. Comment on any outliers, trends,clusters etc (if any).

>E. Plot the loadings of the rst two components and comment on which wavelengths seem to be important for dierent PCs. **Don't** describe the graphs in words. In two sentences, describe any significant trends or signals (if any).

> F. Apply the dimensionality reduction with the PCA model you developed in step B for the **test set** using the **transform** method.

> G. Create a scatter plot of the scores in PC1, PC2 of all 54 points in the train and test data set, coloured by the set they belong to (suggestion: use sns.scatterplot). Why is it important to overlap the scores of both train and test set in one graph? What are
you checking to see?



## 3.3 Task 2: Supervised learning, PLS model
> A. Run the PLS regression to model all 3 outputs with the scaled train set spectra. Increase the number of components to achieve R2 value of at least 0.92 (use PLSRegression's method score to estimate R2).

> B. Predict the output values for the inputs in the test set and compare them by plotting the 24 actual vs predicted values in a 1x3 subplot (for spruce, pine and birch, respectively). Also, include the y=x line for reference.